# Compilation

To be fast, Julia needs to **specialize** code, that is **compile specific native versions of the code** utilizing the type information. **The better the specialization the faster the code!**

## "Just ahead of time" compilation

* Julia **specializes on the types of function arguments** and 
* compiles efficient machine code **when a function is called for the first time** (with these input argument types).

If the same function is called again with the same input argument types, the already existing machine code is reused.


In [2]:
func(x,y) = 2x + y

func (generic function with 1 method)

In [3]:
x = [1.2, 3.4, 5.6] # Vector{Float64}
y = [0.4, 0.7, 0.9] # Vector{Float64}

@time func(x,y);
@time func(x,y);

  0.041341 seconds (186.09 k allocations: 12.677 MiB, 99.97% compilation time)
  0.000002 seconds (2 allocations: 160 bytes)


**First call:** compilation + running the code

**Second call:** running the code


In [4]:
@time func(x,y);

  0.000004 seconds (2 allocations: 160 bytes)


If one of the input types changes, Julia compiles a new specialization of the function!


In [5]:
typeof(x)

Vector{Float64} (alias for Array{Float64, 1})

In [6]:
x = [1, 3, 5]

3-element Vector{Int64}:
 1
 3
 5

In [7]:
typeof(x)

Vector{Int64} (alias for Array{Int64, 1})

In [8]:
@time func(x,y); # Vector{Int64}, Vector{Float64}
@time func(x,y);

  0.043782 seconds (150.96 k allocations: 10.204 MiB, 99.96% compilation time)
  0.000003 seconds (2 allocations: 160 bytes)


We now have two efficient native codes in the cache: one for all `Vector{Float64}` inputs and another one for `Vector{Int64}` as the first and `Vector{Float64}` as the second argument type.

In [9]:
methods(func)

# 1 method for generic function "func" from Main:
 [1] func(x, y)
     @ ~/Julia Courses/JuliaUCL24/notebooks/Day1/2_compilation.ipynb:1

In [10]:
using MethodAnalysis
methodinstances(func)

3-element Vector{Core.MethodInstance}:
 MethodInstance for func(::Any, ::Any)
 MethodInstance for func(::Vector{Float64}, ::Vector{Float64})
 MethodInstance for func(::Vector{Int64}, ::Vector{Float64})

### Compilation pipeline

<p><br><img src="imgs/Julia_compilation_pipeline.svg" width="512"/></p>

* **AST**: abstract syntax tree
* **IR**: intermediate representation

More about Julia compilation, see [Bezanson J et al (2018) Julia: dynamism and performance reconciled by design. Proc ACM Program Lang.](https://doi.org/10.1145/3276490)

### What makes Julia fast?

**Specialization** → (Successful) **Type inference** → **Compilation**

## Introspection tools
#### (*But I really want to see what happens!*)

We can inspect the code at all transformation stages with a bunch of macros:

<img src="./imgs/julia_introspection_macros.svg" width=300px>

In [11]:
@macroexpand @show 3+3

quote
    Base.println("3 + 3 = ", Base.repr(begin
                #= show.jl:1181 =#
                local var"#118#value" = 3 + 3
            end))
    var"#118#value"
end

In [12]:
f(x, y) = x^3 + y/2

f (generic function with 1 method)

In [13]:
@code_lowered f(1.0,2.0)

CodeInfo(
1 ─ %1 = Main.:^
│   %2 = Core.apply_type(Base.Val, 3)
│   %3 = (%2)()
│   %4 = Base.literal_pow(%1, x, %3)
│   %5 = y / 2
│   %6 = %4 + %5
└──      return %6
)

In [14]:
@code_typed f(1.0,2.0)

CodeInfo(
1 ─ %1 = Base.mul_float(x, x)::Float64
│   %2 = Base.mul_float(%1, x)::Float64
│   %3 = Base.div_float(y, 2.0)::Float64
│   %4 = Base.add_float(%2, %3)::Float64
└──      return %4
) => Float64

From the types of the input arguments, Julia has figured out all the intermediate types. This crucial process is known as **type inference** and its success is the basis for a good specialization (i.e. performant native code as a result). Moreover, the generic power function computing the cubic of `x` is replaced by specific floating-point multiplications (**static dispatch**).

In [15]:
@code_llvm debuginfo=:none f(1.0,2.0)

define double @julia_f_1849(double %0, double %1) #0 {
top:
  %2 = fmul double %0, %0
  %3 = fmul double %2, %0
  %4 = fmul double %1, 5.000000e-01
  %5 = fadd double %3, %4
  ret double %5
}


The expensive divide operation (`y/2`) is replaced by multiplying by 0.5. In the end, giving two `Float64` arguments this function has 4 floating-point operations, i.e. 3 multiplications and 1 addition, instead of cubic function and division.

In [16]:
@code_native debuginfo=:none f(1.0,2.0)

	.text
	.file	"f"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_f_1884
.LCPI0_0:
	.quad	0x3fe0000000000000              # double 0.5
	.text
	.globl	julia_f_1884
	.p2align	4, 0x90
	.type	julia_f_1884,@function
julia_f_1884:                           # @julia_f_1884
# %bb.0:                                # %top
	push	rbp
	vmulsd	xmm2, xmm0, xmm0
	movabs	rax, offset .LCPI0_0
	mov	rbp, rsp
	vmulsd	xmm1, xmm1, qword ptr [rax]
	vmulsd	xmm0, xmm2, xmm0
	vaddsd	xmm0, xmm0, xmm1
	pop	rbp
	ret
.Lfunc_end0:
	.size	julia_f_1884, .Lfunc_end0-julia_f_1884
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


Let's compare this to integer inputs.

In [17]:
@code_native debuginfo=:none f(1,2)

	.text
	.file	"f"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_f_1888
.LCPI0_0:
	.quad	0x3fe0000000000000              # double 0.5
	.text
	.globl	julia_f_1888
	.p2align	4, 0x90
	.type	julia_f_1888,@function
julia_f_1888:                           # @julia_f_1888
# %bb.0:                                # %top
	push	rbp
	mov	rcx, rdi
	vcvtsi2sd	xmm0, xmm0, rsi
	movabs	rax, offset .LCPI0_0
	mov	rbp, rsp
	imul	rcx, rdi
	vmulsd	xmm0, xmm0, qword ptr [rax]
	imul	rcx, rdi
	vcvtsi2sd	xmm1, xmm1, rcx
	vaddsd	xmm0, xmm0, xmm1
	pop	rbp
	ret
.Lfunc_end0:
	.size	julia_f_1888, .Lfunc_end0-julia_f_1888
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


### Recommendation: [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl)
While these introspection macros are great, we recommend to use `@descend` from the package [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl) for real world code analysis.

Essentially, Cthulhu is an **interactive**, more powerful generalization of the macros above.

* Allows easy switching between code representations (syntax, typed, native, ...).
* **Recursive application possible**(!) (i.e. introspecting a function that is called within a function within function ...).

However, due to its interactivity, it doesn't work in Jupyter but **only works in the REPL** (→ exercise).

<img src="./imgs/cthulhu.png" width=1000>

## How important is specialization?

Let's try to estimate the performance gain by specialization.

To prevent specialization, we deliberately throw away any useful type information and operate on a `Vector{Any}` that can literally store anything!

(This is qualitatively comparable to what Python does.)


In [18]:
func(v) = 2*v[1] + v[2] # version of func that takes in a vector

func (generic function with 2 methods)

In [19]:
rand(2)

2-element Vector{Float64}:
 0.927918719389245
 0.1934154611149782

In [20]:
Any[rand(), rand()]

2-element Vector{Any}:
 0.1757754666614454
 0.7986265068547347

For benchmarking we will use `@btime` (or `@benchmark`) from [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl). This will take care of a couple of things for us:
* Exclude first run.
* Run the code multiple times (→ statistics).
* Benchmark in a function (local scope).

**General rule:** For proper benchmarking don't use `@time` but `@btime` and interpolate (`$`) global input arguments.

(Prefixing variable with `$` always means interpolation in Julia, e.g. string interpolation.)

In [21]:
ucl = "UCL ARC"
welcome = "Welcome to $ucl"

"Welcome to UCL ARC"

In [22]:
using BenchmarkTools

v_typed = rand(2)
v_any = Any[rand(), rand()]

@btime func($v_typed);
@btime func($v_any);

  1.643 ns (0 allocations: 0 bytes)


  18.747 ns (2 allocations: 32 bytes)


In [23]:
@benchmark func($v_any)

BenchmarkTools.Trial: 10000 samples with 998 evaluations.
 Range (min … max):  17.202 ns … 393.214 ns  ┊ GC (min … max): 0.00% … 92.16%
 Time  (median):     18.626 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.249 ns ±   9.027 ns  ┊ GC (mean ± σ):  1.18% ±  2.42%

        ▃▃▅█▇▂                                                  
  ▂▂▂▃▅███████▆▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂ ▃
  17.2 ns         Histogram: frequency by time         26.1 ns <

 Memory estimate: 32 bytes, allocs estimate: 2.

In [24]:
@code_typed func(rand(2))

CodeInfo(
1 ─ %1 = Base.arrayref(true, v, 1)::Float64
│   %2 = Base.mul_float(2.0, %1)::Float64
│   %3 = Base.arrayref(true, v, 2)::Float64
│   %4 = Base.add_float(%2, %3)::Float64
└──      return %4
) => Float64

**static dispatch**: the generic functions `*` and `+` are replaced by specific implementations.

In [25]:
@code_typed func(Any[rand(), rand()])

CodeInfo(
1 ─ %1 = Base.arrayref(true, v, 1)::Any
│   %2 = (2 * %1)::Any
│   %3 = Base.arrayref(true, v, 2)::Any
│   %4 = (%2 + %3)::Any
└──      return %4
) => Any

Note here the generic functions `*` and `+` can not be replaced by specific variants due to lack of type information. This leads to inefficient **runtime dispatch**.

## Dispatch and specialization

**Types drive both dispatch and specialization.**

First, the most specific method is selected (dispatch), then it gets compiled to efficient native code (specialization). Let's reconsider our earlier example:

In [26]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [27]:
@code_native myabs(3.2 + 4.5im) # complex input

	.text
	.file	"myabs"
	.globl	julia_myabs_2668                # -- Begin function julia_myabs_2668
	.p2align	4, 0x90
	.type	julia_myabs_2668,@function
julia_myabs_2668:                       # @julia_myabs_2668
; ┌ @ /home/javier/Julia Courses/JuliaUCL24/notebooks/Day1/2_compilation.ipynb:2 within `myabs`
# %bb.0:                                # %L13
	push	rbp
; │┌ @ complex.jl:296 within `*` @ float.jl:411
	vmovupd	xmm0, xmmword ptr [rdi]
	mov	rbp, rsp
	vmulpd	xmm0, xmm0, xmm0
; ││ @ complex.jl:296 within `*`
; ││┌ @ float.jl:410 within `-`
	vpermilpd	xmm1, xmm0, 1           # xmm1 = xmm0[1,0]
	vaddsd	xmm0, xmm0, xmm1
; │└└
; │┌ @ math.jl:687 within `sqrt`
	vsqrtsd	xmm0, xmm0, xmm0
; │└
	pop	rbp
	ret
.Lfunc_end0:
	.size	julia_myabs_2668, .Lfunc_end0-julia_myabs_2668
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits


In [28]:
@code_native myabs(3 + 4im) # also complex input but different native code (due to specialization)!

	.text
	.file	"myabs"
	.globl	julia_myabs_2675                # -- Begin function julia_myabs_2675
	.p2align	4, 0x90
	.type	julia_myabs_2675,@function
julia_myabs_2675:                       # @julia_myabs_2675
; ┌ @ /home/javier/Julia Courses/JuliaUCL24/notebooks/Day1/2_compilation.ipynb:2 within `myabs`
# %bb.0:                                # %top
	push	rbp
; │┌ @ complex.jl:296 within `*` @ int.jl:88
	mov	rax, qword ptr [rdi]
; │└
; │┌ @ complex.jl:282 within `conj`
; ││┌ @ int.jl:85 within `-`
	mov	rcx, qword ptr [rdi + 8]
	mov	rbp, rsp
; │└└
; │┌ @ complex.jl:296 within `*` @ int.jl:88
	imul	rax, rax
	imul	rcx, rcx
; ││ @ complex.jl:296 within `*`
; ││┌ @ int.jl:86 within `-`
	add	rcx, rax
; │└└
; │┌ @ math.jl:1576 within `sqrt`
; ││┌ @ float.jl:294 within `float`
; │││┌ @ float.jl:268 within `AbstractFloat`
; ││││┌ @ float.jl:159 within `Float64`
	vcvtsi2sd	xmm0, xmm0, rcx
; ││└└└
; ││ @ math.jl:1578 within `sqrt` @ math.jl:686
	js	.LBB0_2
# %bb.1:                              

## Are explicit type annotations necessary? (think C or Fortran)

Note that Julia's type inference is powerful. Specifying types **is not** necessary for best performance!


In [29]:
function my_function(x)
    y = rand()
    z = rand()
    x+y+z
end

function my_function_typed(x::Int64)::Float64
    y::Float64 = rand()
    z::Float64 = rand()
    x+y+z
end

my_function_typed (generic function with 1 method)

In [30]:
@btime my_function(10);
@btime my_function_typed(10);

  2.251 ns (0 allocations: 0 bytes)


  2.254 ns (0 allocations: 0 bytes)


Annotating types explicitly can serve a purpose.

* Enforce conversions
* Rather rarely: help the compiler infer types in tricky situations

However, more often than not it is an indication of suboptimal code design. (It also makes functions much less generic and reusable!)

## Compilation on heterogeneous HPC clusters

By default, Julia produces native code for the CPU type it is running on. This means that it uses the [Instruction Set Architecture (ISA)](https://en.wikipedia.org/wiki/Instruction_set_architecture) of this CPU.

This can lead to issues on heterogeneous clusters where different nodes have different CPU types. E.g. you precompile Julia packages on a login node with an Intel CPU but want to run the code on a compute node with AMD CPUs.

**Solution: Multiversioning**

```julia
# Noctua 1 & 2
export JULIA_CPU_TARGET="generic;znver3,clone_all;skylake,clone_all"

# Noctua 2 & DGX
export JULIA_CPU_TARGET="generic;znver3,clone_all;znver2,clone_all"
```

This will compile a generic (but slow) variant as well as efficient variants for AMD Zen3 and Intel Skylake CPUs / AMD Zen2.

# Core messages of this Notebook

* **A function is compiled when called for the first time** with a given set of argument types.
* There are **multiple code transformation steps** which can be inspected through macros like `@code_warntype` or `@descend` from Cthulhu.jl.
* What makes Julia fast? Successful **Type inference** → **Specialization** → **Compilation**.
* Functions should almost always be benchmarked with **BenchmarkTools.jl's `@btime` and `@benchmark`** instead of `@time`.
* In virtually all cases, **explicit type annotations are irrelevant for performance**.